# Chatea con modelos de visión

**Si estás buscando la aplicación web, revisa la carpeta src/.**

Este notebook solo se proporciona para experimentar manualmente con el modelo de visión.

## Autentícate en OpenAI

El siguiente código se conecta a OpenAI, ya sea usando una cuenta de Azure OpenAI, modelos de GitHub, o modelo local de Ollama. Mira el README para instrucciones sobre cómo configurar el archivo `.env`.

In [ ]:
import os

import azure.identity
import openai
from dotenv import load_dotenv

load_dotenv(".env", override=True)

openai_host = os.getenv("OPENAI_HOST", "github")

if openai_host == "github":
    print("Usando GitHub Models con GITHUB_TOKEN como clave")
    openai_client = openai.OpenAI(
        api_key=os.environ["GITHUB_TOKEN"],
        base_url="https://models.github.ai/inference",
    )
    model_name = os.getenv("OPENAI_MODEL", "openai/gpt-4o")
elif openai_host == "local":
    print("Usando API local compatible con OpenAI sin clave")
    openai_client = openai.OpenAI(api_key="no-key-required", base_url=os.environ["LOCAL_OPENAI_ENDPOINT"])
    model_name = os.getenv("OPENAI_MODEL", "gpt-4o")
elif openai_host == "azure" and os.getenv("AZURE_OPENAI_KEY_FOR_CHATVISION"):
    # Autenticación usando una clave de API de Azure OpenAI
    # Esto generalmente no se recomienda, pero se proporciona por conveniencia
    print("Usando Azure OpenAI con clave")
    openai_client = openai.OpenAI(
        base_url=os.environ["AZURE_OPENAI_ENDPOINT"] + "/openai/v1/",
        api_key=os.environ["AZURE_OPENAI_KEY_FOR_CHATVISION"],
    )
    # Esto es en realidad el nombre del deployment, no el nombre del modelo
    model_name = os.getenv("OPENAI_MODEL", "gpt-4o")
elif openai_host == "azure" and os.getenv("AZURE_OPENAI_ENDPOINT"):
    tenant_id = os.environ["AZURE_TENANT_ID"]
    print("Usando Azure OpenAI con credencial de Azure Developer CLI")
    default_credential = azure.identity.AzureDeveloperCliCredential(tenant_id=tenant_id)
    token_provider = azure.identity.get_bearer_token_provider(
        default_credential, "https://cognitiveservices.azure.com/.default"
    )
    openai_client = openai.OpenAI(
        base_url=os.environ["AZURE_OPENAI_ENDPOINT"] + "/openai/v1/",
        api_key=token_provider,
    )
    # Esto es en realidad el nombre del deployment, no el nombre del modelo
    model_name = os.getenv("OPENAI_MODEL", "gpt-4o")

print(f"Usando modelo {model_name}")

## Envía una imagen por URL

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {"text": "¿es un unicornio?", "type": "text"},
            {
                "image_url": {"url": "https://upload.wikimedia.org/wikipedia/commons/6/6e/Ur-painting.jpg"},
                "type": "image_url",
            },
        ],
    }
]
response = openai_client.chat.completions.create(model=model_name, messages=messages)

print(response.choices[0].message.content)

## Envía una imagen por Data URI



In [ ]:
import base64


def open_image_as_base64(filename):
    with open(filename, "rb") as image_file:
        image_data = image_file.read()
    image_base64 = base64.b64encode(image_data).decode("utf-8")
    return f"data:image/png;base64,{image_base64}"

In [ ]:
response = openai_client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": "¿estos son cocodrilos o caimanes?", "type": "text"},
                {"image_url": {"url": open_image_as_base64("../mystery_reptile.png")}, "type": "image_url"},
            ],
        }
    ],
)

print(response.choices[0].message.content)

## Casos de uso para análisis de imágenes

### Accesibilidad

#### Asistencia para personas con discapacidad visual

In [ ]:
response = openai_client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": "¿hay algo bueno para veganos en este menú?", "type": "text"},
                {"image_url": {"url": open_image_as_base64("../menu.png")}, "type": "image_url"},
            ],
        }
    ],
)

print(response.choices[0].message.content)

#### Subtítulos automáticos de imágenes

In [ ]:
response = openai_client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": "Sugiere un texto alternativo para esta imagen", "type": "text"},
                {"image_url": {"url": open_image_as_base64("../azure_arch.png")}, "type": "image_url"},
            ],
        }
    ],
)

print(response.choices[0].message.content)

### Automatización de procesos de negocio

#### Procesamiento de reclamaciones de seguros

In [ ]:
response = openai_client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "system",
            "content": (
                "Eres un asistente de IA que ayuda a las compañías de seguros de autos a procesar reclamaciones. "
                "Aceptas imágenes de autos dañados que se envían con las reclamaciones, y puedes hacer juicios "
                "sobre las causas del daño automovilístico y la validez de las reclamaciones sobre ese daño."
            ),
        },
        {
            "role": "user",
            "content": [
                {"text": "La reclamación dice que este daño es por granizo. ¿Es válida?", "type": "text"},
                {"image_url": {"url": open_image_as_base64("../dented_car.jpg")}, "type": "image_url"},
            ],
        },
    ],
)

print(response.choices[0].message.content)

#### Análisis de gráficos

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {"text": "¿En qué zona estamos perdiendo más árboles?", "type": "text"},
            {
                "image_url": {
                    "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/1/1f/20210331_Global_tree_cover_loss_-_World_Resources_Institute.svg/1280px-20210331_Global_tree_cover_loss_-_World_Resources_Institute.svg.png"
                },
                "type": "image_url",
            },
        ],
    }
]
response = openai_client.chat.completions.create(model=os.environ["OPENAI_MODEL"], messages=messages)

print(response.choices[0].message.content)

#### Análisis de tablas

In [ ]:
response = openai_client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": "¿Cuál es la planta más barata?", "type": "text"},
                {"image_url": {"url": open_image_as_base64("../page_0.png")}, "type": "image_url"},
            ],
        }
    ],
)

print(response.choices[0].message.content)

#### Soporte de electrodomésticos

In [ ]:
response = openai_client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": "¿Cómo configuro esto para lavar los platos rápidamente?", "type": "text"},
                {"image_url": {"url": open_image_as_base64("../dishwasher.png")}, "type": "image_url"},
            ],
        }
    ],
)

print(response.choices[0].message.content)